In [1]:
import string
import copy
import time
%matplotlib inline
import os
# os.listdir("../input/")
import torch
import torch.nn as nn
import pandas as pd
import numpy as np
os.system('pip install pytorch_pretrained_bert --no-index --find-links="../input/pytorch-pretrained-bert/pytorch_pretrained_bert" ')
from torch.utils.data import Dataset, TensorDataset, DataLoader, RandomSampler, SequentialSampler
from transformers.models.bert.modeling_bert import BertPreTrainedModel, BertModel
from transformers import BertTokenizer
from transformers import BertForSequenceClassification, AdamW, BertConfig
import random
from transformers import get_linear_schedule_with_warmup
from torch.cuda.amp import GradScaler, autocast
import warnings
warnings.filterwarnings("ignore")
import matplotlib.pyplot as plt
from torch.utils.tensorboard import SummaryWriter

SEED = 42
BATCH_SIZE = 32
MAX_LENGTH = 356
BERT_FP = '../input/bert-base-uncased'
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
n_gpu = torch.cuda.device_count()

random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
if n_gpu > 0:
    torch.cuda.manual_seed_all(SEED)

In [2]:
import datetime

def format_time(elapsed):
    '''
    Takes a time in seconds and returns a string hh:mm:ss
    '''
    # Round to the nearest second.
    elapsed_rounded = int(round((elapsed)))
    
    # Format as hh:mm:ss
    return str(datetime.timedelta(seconds=elapsed_rounded))

In [3]:
def children(m):
    return m if isinstance(m, (list, tuple)) else list(m.children())


def set_trainable_attr(m, b):
    m.trainable = b
    for p in m.parameters():
        p.requires_grad = b


def apply_leaf(m, f):
    c = children(m)
    if isinstance(m, nn.Module):
        f(m)
    if len(c) > 0:
        for l in c:
            apply_leaf(l, f)


def set_trainable(l, b):
    apply_leaf(l, lambda m: set_trainable_attr(m, b))

In [4]:
class BertForSequenceRegression(nn.Module):
    def __init__(self):
        super(BertForSequenceRegression, self).__init__()
        self.bert = BertModel.from_pretrained(BERT_FP)
        self.dropout = nn.Dropout(0.1)
        self.linear1 = nn.Linear(768, 256)
        self.relu = nn.ReLU()
        self.linear2 = nn.Linear(256, 1)

    def forward(self, ids,  token_type_ids, attention_mask):
        _, pooled_output = self.bert(ids, token_type_ids, attention_mask, return_dict=False)
        pooled_output = self.dropout(pooled_output)
        pooled_output = self.linear1(pooled_output)
        pooled_output = self.relu(pooled_output)
        outputs = self.linear2(pooled_output)
        return outputs.view(-1)

In [5]:
def RMSELoss(outputs, targets):
    return torch.sqrt(nn.MSELoss()(outputs, targets))

In [6]:
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.preprocessing import KBinsDiscretizer

class regressor_stratified_cv:
    def __init__(self, n_splits = 10, n_repeats = 2, group_count = 10,
                 random_state = 0, strategy = 'quantile'):
        self.group_count = group_count
        self.strategy = strategy
        self.cvkwargs = dict(n_splits = n_splits, n_repeats = n_repeats, 
                             random_state = random_state)
        self.cv = RepeatedStratifiedKFold(**self.cvkwargs)
        self.discretizer = KBinsDiscretizer(n_bins = self.group_count, encode = 'ordinal',
                                            strategy = self.strategy)  
            
    def split(self, X, y, groups = None):
        kgroups=self.discretizer.fit_transform(y[:, None])[:, 0]
        return self.cv.split(X, kgroups, groups)
    
    def get_n_splits(self, X, y, groups = None):
        return self.cv.get_n_splits(X, y, groups)

In [7]:
def text_preprocessing(excerpt):
    
    # lower casing
    excerpt = excerpt.lower()

    # removal of punctuation
    excerpt = excerpt.translate(str.maketrans('', '', string.punctuation))

        
    # removal of stopwords
#     from nltk.corpus import stopwords
#     ", ".join(stopwords.words('english'))
#     STOPWORDS = set(stopwords.words('english'))
#     excerpt = " ".join([word for word in str(excerpt).split() if word not in STOPWORDS])
        
    # lemmatization 
#     from nltk.stem import WordNetLemmatizer
#     lemmatizer = WordNetLemmatizer()
#     excerpt = " ".join([lemmatizer.lemmatize(word) for word in excerpt.split()])
        
                
    return excerpt

In [8]:
df = pd.read_csv('../input/commonlitreadabilityprize/train.csv')
df

,id,url_legal,license,excerpt,target,standard_error
0,c12129c31,NaN,NaN,When the young people returned to the ballroom...,-0.340259,0.464009
1,85aa80a4c,NaN,NaN,"All through dinner time, Mrs. Fayre was somewh...",-0.315372,0.480805
2,b69ac6792,NaN,NaN,"As Roger had predicted, the snow departed as q...",-0.580118,0.476676
3,dd1000b26,NaN,NaN,And outside before the palace a great garden w...,-1.054013,0.450007
4,37c1b32fb,NaN,NaN,Once upon a time there were Three Bears who li...,0.247197,0.510845
...,...,...,...,...,...,...
2829,25ca8f498,https://sites.ehe.osu.edu/beyondpenguins/files...,CC BY-SA 3.0,When you think of dinosaurs and where they liv...,1.711390,0.646900
2830,2c26db523,https://en.wikibooks.org/wiki/Wikijunior:The_E...,CC BY-SA 3.0,So what is a solid? Solids are usually hard be...,0.189476,0.535648
2831,cd19e2350,https://en.wikibooks.org/wiki/Wikijunior:The_E...,CC BY-SA 3.0,The second state of matter we will discuss is ...,0.255209,0.483866
2832,15e2e9e7a,https://en.wikibooks.org/wiki/Geometry_for_Ele...,CC BY-SA 3.0,Solids are shapes that you can actually touch....,-0.215279,0.514128


In [9]:
df["text"] = df["excerpt"].apply(lambda x: text_preprocessing(x))

In [10]:
excerpts = df.text.values
targets = df.target.values

In [11]:
# Load the BERT tokenizer.
tokenizer = BertTokenizer.from_pretrained(BERT_FP, do_lower_case=True)

In [12]:
class TokenDataset(Dataset):
    def __init__(self, tokenizer, text, target = None, is_test=False):
        self.text = text
        self.target = target
        self.is_test = is_test
        self.max_len = MAX_LENGTH
        self.tokenizer = tokenizer
        
    def __len__(self):
        return len(self.text)
    
    def __getitem__(self, idx):
        text = str(self.text[idx])
        text = ' '.join(text.split())
        inputs = self.tokenizer.encode_plus(
            text,
            None,
            truncation=True,
            add_special_tokens=True,
            max_length = self.max_len,
            padding='max_length',
            return_token_type_ids=True
        )
        ids = torch.tensor(inputs['input_ids'], dtype=torch.long)
        mask = torch.tensor(inputs['attention_mask'], dtype=torch.long)
        token_type_ids = torch.tensor(inputs['token_type_ids'], dtype=torch.long)
        
        if self.is_test:
            return {
                'ids': ids,
                'mask': mask,
                'token_type_ids': token_type_ids,
            }
        else:    
            targets = torch.tensor(self.target[idx], dtype=torch.float)
            return {
                'ids': ids,
                'mask': mask,
                'token_type_ids': token_type_ids,
                'targets': targets
            }

In [13]:
writer = SummaryWriter()

In [14]:
n_splits = 5
n_repeats = 1
group_count = 10

cv = regressor_stratified_cv(n_splits = n_splits, n_repeats = n_repeats,
                           group_count = group_count, random_state = 0, strategy = 'quantile')

df = df[['text', 'target']]

epochs = 10
n_epochs_stop = 5
epochs_no_improve = 0
training_stats = []
i = 1
eval_losses = []
scaler = GradScaler()
# input_ids, attention_masks, token_type_ids = encode(excerpts, tokenizer)
for train_idx, test_idx in cv.split(df["text"], df["target"].values):
    train_data = df.loc[train_idx]
    test_data = df.loc[test_idx]    

In [15]:
train_set = TokenDataset(tokenizer,
                        text = train_data['text'].values,
                        target = train_data['target'].values
                        )
    
test_set = TokenDataset(tokenizer,
                        text = test_data['text'].values,
                        target = test_data['target'].values
                        )

train_dataloader = DataLoader(train_set, batch_size = BATCH_SIZE, shuffle = True, num_workers=8)

test_dataloader = DataLoader(test_set, batch_size = BATCH_SIZE, shuffle = True, num_workers=8)
    
model = BertForSequenceRegression().to(device)
model.load_state_dict(torch.load('../input/model-4-20-0649/model_fold_4_epoch_20_loss_0.649.pt'))
set_trainable(model, True)
set_trainable(model.bert.embeddings, True)
set_trainable(model.bert.encoder, True)
# Defining Optimizer with weight decay to params other than bias and layer norms
param_optimizer = list(model.named_parameters())
no_decay = ["bias", "LayerNorm.bias", "LayerNorm.weight"]
optimizer_parameters = [
    {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)], 
    'weight_decay': 0.001},
    {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)], 
    'weight_decay': 0.0}
]  
optimizer = AdamW(optimizer_parameters,
                      lr = 2e-6
                     )
total_steps = (len(train_dataloader) * epochs)               
#     num_steps = int(total_steps * 0.1)
scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps = 0,
                                            num_training_steps = total_steps)
iter_eval_loss = []
min_eval_loss = np.Inf
for epoch_i in range(0, epochs):
    print("")
    print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
    t0 = time.time()
    # training
    model.train()
    tr_loss = []
    for step, batch in enumerate(train_dataloader):
        ids = batch['ids'].to(device, dtype=torch.long)
        input_mask = batch['mask'].to(device, dtype=torch.long)
        type_ids = batch['token_type_ids'].to(device, dtype=torch.long)
        target = batch['targets'].to(device, dtype=torch.float)
        optimizer.zero_grad()
        with autocast():
            output = model(ids, input_mask, type_ids)
            loss = RMSELoss(output, target)
            tr_loss.append(loss.cpu().detach().numpy().tolist())

        scheduler.step()
        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()
            
    train_losses = np.mean(tr_loss)  
    training_time = format_time(time.time() - t0)
    writer.add_scalar(f"Loss/train_fold_{i}_epoch_{epoch_i+1}", train_losses, epoch_i)
    print("Train loss: ", train_losses)
    print("Training epcoh took: {:}".format(training_time))
        
    # evaluation
    t0 = time.time()
    all_targets, all_preds = [], []
    model.eval()   
    eval_loss = []
        
    # disable gradients 
    with torch.no_grad(): 
        for batch in test_dataloader:
            ids = batch['ids'].to(device, dtype=torch.long)
            input_mask = batch['mask'].to(device, dtype=torch.long)
            type_ids = batch['token_type_ids'].to(device, dtype=torch.long)
            target = batch['targets'].to(device, dtype=torch.float)
            output = model(ids, input_mask, type_ids)
            loss = RMSELoss(output, target)
            eval_loss.append(loss.cpu().detach().numpy().tolist())
            
    epoch_eval_loss = np.mean(eval_loss)
    eval_time = format_time(time.time() - t0)
    writer.add_scalar(f"Loss/eval_fold_{i}_epoch_{epoch_i+1}", epoch_eval_loss, epoch_i)
    print("Eval loss: ", epoch_eval_loss)
    print("Evaluation took: {:}".format(eval_time))
        
    # recording all statistics from this epoch
    training_stats.append({
            'fold' : i,
            'epoch': epoch_i + 1,
            'Training Loss': train_losses,
            'Eval Loss': epoch_eval_loss,
            'Training Time': training_time,
            'Eval Time': eval_time
    })
        
    # early stopping and saving best model
    if epoch_eval_loss < min_eval_loss:
        min_eval_loss = epoch_eval_loss
        best_model = copy.deepcopy(model)
        PATH = f'model_epoch_{epoch_i+1}_loss_{round(epoch_eval_loss, 3)}.pt'
    
torch.save(best_model.state_dict(), PATH)   
torch.cuda.empty_cache()
writer.flush()
writer.close()


======== Epoch 1 / 10 ========
Train loss:  0.6145947113843031
Training epcoh took: 0:01:28
Eval loss:  0.6512783500883315
Evaluation took: 0:00:08

======== Epoch 2 / 10 ========
Train loss:  0.581712223694358
Training epcoh took: 0:01:27
Eval loss:  0.6650660269790225
Evaluation took: 0:00:09

======== Epoch 3 / 10 ========
Train loss:  0.5491588002359363
Training epcoh took: 0:01:27
Eval loss:  0.6579830629958047
Evaluation took: 0:00:08

======== Epoch 4 / 10 ========
Train loss:  0.535218789124153
Training epcoh took: 0:01:27
Eval loss:  0.6403369224733777
Evaluation took: 0:00:08

======== Epoch 5 / 10 ========
Train loss:  0.5161133123115754
Training epcoh took: 0:01:27
Eval loss:  0.6299225903219647
Evaluation took: 0:00:08

======== Epoch 6 / 10 ========
Train loss:  0.5023194517887813
Training epcoh took: 0:01:27
Eval loss:  0.6257989754279455
Evaluation took: 0:00:08

======== Epoch 7 / 10 ========
Train loss:  0.4949530048269621
Training epcoh took: 0:01:27
Eval loss:  0.6

In [16]:
# n_splits = 5
# n_repeats = 2
# group_count = 10
# cv = regressor_stratified_cv(n_splits = n_splits, n_repeats = n_repeats,
#                            group_count = group_count, random_state = 0, strategy = 'quantile')

# for train_index, test_index in cv.split(input_ids, targets):
#     train_inputs, test_inputs = input_ids[train_index], input_ids[test_index]
#     train_targets, test_targets = targets[train_index], targets[test_index]
#     train_masks, test_masks = attention_masks[train_index], attention_masks[test_index]
#     train_type_ids, test_type_ids = token_type_ids[train_index], token_type_ids[test_index]

In [17]:
# set_trainable(model, True)
# set_trainable(model.bert.embeddings, True)    
# set_trainable(model.bert.encoder, True)

In [18]:
# epochs = 5
# optimizer = AdamW(model.parameters(),
#                   lr = 5e-6,
#                   eps = 1e-6 
#                 )
# total_steps = len(train_dataloader) * epochs
# scheduler = get_linear_schedule_with_warmup(optimizer, 
#                                             num_warmup_steps = 0,
#                                             num_training_steps = total_steps)
# eval_losses = []
# for epoch_i in range(0, epochs):
#     print("")
#     print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
#     # training
#     model.train()
#     tr_loss = []
    
#     for step, batch in enumerate(train_dataloader):
#         batch = tuple(t.to(device) for t in batch)
#         ids, input_mask, type_ids, target = batch
#         output = model(ids, input_mask, type_ids, target)
#         loss = RMSELoss(output, target)
#         tr_loss.append(loss.cpu().detach().numpy().tolist())
#         loss.backward()  
#         optimizer.step()
#         optimizer.zero_grad()  
#         scheduler.step()
            
#     train_losses = np.mean(tr_loss)  
#     print("Train loss: ", train_losses)
#     all_targets, all_preds = [], []
#     model.eval()   
#     eval_loss = []
#     # evaluation
#     # disable gradients 
#     with torch.no_grad(): 
#         for batch in test_dataloader:
#             batch = tuple(t.to(device) for t in batch)
#             ids, input_mask, type_ids, target = batch
#             output = model(ids, input_mask, type_ids, target)
#             loss = RMSELoss(output, target)
#         eval_loss.append(loss.cpu().detach().numpy().tolist())
            
#     epoch_eval_loss = np.mean(eval_loss)
#     print("Eval loss: ", epoch_eval_loss)

#     eval_losses.append(epoch_eval_loss)   
# torch.cuda.empty_cache()
# mean_eval_loss = np.mean(eval_losses)
# print(mean_eval_loss)

In [19]:
# test = pd.read_csv('../input/commonlitreadabilityprize/test.csv')

In [20]:
# test["text"] = test["excerpt"].apply(lambda x: text_preprocessing(x))

In [21]:
# excerpts = test.text.values

In [22]:
# test_data = test[['text']]

In [23]:
# test_set = TokenDataset(tokenizer,
#                         text = test_data['text'].values, is_test = True
#                         )

In [24]:
# test_dataloader = DataLoader(test_set, batch_size = BATCH_SIZE, num_workers=8)

In [25]:
# model = BertForSequenceRegression().to(device)
# model.load_state_dict(torch.load('../input/model-1-13-0662/model_fold_1_epoch_13_loss_0.662.pt'))

In [26]:
# predictions = []
# # disable gradients 
# with torch.no_grad(): 
#     for batch in test_dataloader:
#         ids = batch['ids'].to(device, dtype=torch.long)
#         input_mask = batch['mask'].to(device, dtype=torch.long)
#         type_ids = batch['token_type_ids'].to(device, dtype=torch.long)
#         output = model(ids, input_mask, type_ids)
#         output = output.cpu().detach().numpy().tolist()
#         predictions += output

In [27]:
# submission = pd.DataFrame({'id':test['id'],'target':predictions})

In [28]:
# submission.to_csv('submission.csv',index=False)

In [29]:
# torch.cuda.empty_cache()